<a href="https://colab.research.google.com/github/danielsaggau/IR_LDC/blob/main/model/SCOTUS/scotus_simcse_base_conversion_longformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/danielsaggau/IR_LDC.git

In [ ]:
%cd IR_LDC 

In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install huggingface_hub
!python -c "from huggingface_hub.hf_api import HfFolder; HfFolder.save_token('XXXX')"

In [ ]:
!python /content/IR_LDC/model/Longformer/Longformer/convert_bert_to_longformer.py # changed the checkpoint manually

In [8]:
from torch.utils.data import DataLoader
import math
from sentence_transformers import models, losses
from sentence_transformers import LoggingHandler, SentenceTransformer, InputExample
import logging
from datetime import datetime
import gzip
import sys
import tqdm

In [9]:
from datasets import load_dataset
dataset = load_dataset("lex_glue", "scotus")

Generating train split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1400 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1400 [00:00<?, ? examples/s]

Dataset lex_glue downloaded and prepared to /root/.cache/huggingface/datasets/lex_glue/scotus/1.0.0/8a66420941bf6e77a7ddd4da4d3bfb7ba88ef48c1d55302a568ac650a095ca3a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
with open('your_file.txt', 'w') as f:
    for line in dataset['train']['text']:
        f.write(f"{line}\n")

In [16]:
# Training parameters
access="XXXX"
model_name = '/content/IR_LDC/models/legal-longformer'
train_batch_size = 4
max_seq_length = 4096
num_epochs = 8

In [17]:
#Input file path (a text file, each line a sentence)
if len(sys.argv) < 2:
    print("Run this script with: python {} path/to/sentences.txt".format(sys.argv[0]))
    exit()

#filepath = sys.argv[1]
filepath = "/content/IR_LDC/your_file.txt"

# Save path to store our model
output_name = ''
if len(sys.argv) >= 3:
    output_name = "-"+sys.argv[2].replace(" ", "_").replace("/", "_").replace("\\", "_")

model_output_path = 'output/train_simcse{}-{}'.format(output_name, datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))

# Use Huggingface/transformers model (like BERT, RoBERTa, XLNet, XLM-R) for mapping tokens to embeddings
model = SentenceTransformer(model_name, use_auth_token=access)

################# Read the train corpus  #################
train_samples = []
with gzip.open(filepath, 'rt', encoding='utf8') if filepath.endswith('.gz') else open(filepath, encoding='utf8') as fIn:
    for line in tqdm.tqdm(fIn, desc='Read file'):
        line = line.strip()
        if len(line) >= 10:
            train_samples.append(InputExample(texts=[line, line]))

# We train our model using the MultipleNegativesRankingLoss
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size, drop_last=True)
train_loss = losses.MultipleNegativesRankingLoss(model)

warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)  # 10% of train data for warm-up

Some weights of the model checkpoint at /content/IR_LDC/models/legal-longformer were not used when initializing LongformerModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerModel were not initialized from the model checkpoint at /content/IR_LDC/models/legal-longformer and are newly initialized: ['longformer.pooler.dense.bias', 'longformer.pooler.dense.weight']
You should probably T

In [18]:
from sentence_transformers import SentenceTransformer, models
word_embedding_model = models.Transformer(model_name)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), 'mean') # remove this block to do mean pooling
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

Some weights of the model checkpoint at /content/IR_LDC/models/legal-longformer were not used when initializing LongformerModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerModel were not initialized from the model checkpoint at /content/IR_LDC/models/legal-longformer and are newly initialized: ['longformer.pooler.dense.bias', 'longformer.pooler.dense.weight']
You should probably T

In [20]:
# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=8,
          warmup_steps=warmup_steps,
          evaluation_steps=500,
          steps_per_epoch=5000,
          callback="epoch",
          output_path='/content/drive/MyDrive/SIMCSE_SCOTUS_base',
          optimizer_params={'lr': 3e-5},
          checkpoint_path='/content/drive/MyDrive/SIMCSE_SCOTUS/output_base',
          show_progress_bar=True,
          checkpoint_save_steps=8000,
          save_best_model=True,
          use_amp=True  # Set to True, if your GPU supports FP16 cores
          )

OutOfMemoryError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')